<a href="https://colab.research.google.com/github/Anil-matcha/Free-Video-Tools/blob/main/Hardcode_subtitles_on_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Install Required Libraries

In [1]:
!pip install moviepy pysrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=f195fb1c3e52a37f2fab3195aa9c3cd0b7df8ab71b4bb9dc16ca401cddb882ba
  Stored in directory: /root/.cache/pip/wheels/30/7f/e8/55de9a9b07302d9e7fe47c27910e3bea0c48536153e74bd7e6
Successfully built pysrt


Setup ImageMagick

In [2]:
!apt install imagemagick &> /dev/null
!sed -i '/<policy domain="path" rights="none" pattern="@\*"/d' /etc/ImageMagick-6/policy.xml
!sudo sed -i 's/<policy domain="resource" name="width" value="[0-9]*KP"\/>/<policy domain="resource" name="width" value="16384K"\/>/' /etc/ImageMagick-6/policy.xml
!sudo sed -i 's/<policy domain="resource" name="height" value="[0-9]*KP"\/>/<policy domain="resource" name="height" value="16384K"\/>/' /etc/ImageMagick-6/policy.xml

Step 2: Import Libraries

Let's import the libraries we need.



In [14]:
import moviepy.editor as mp
import pysrt
from IPython.display import HTML

Step 3: Define Functions for Adding Subtitles

3.1 Function: srt_to_moviepy_subtitles

This function converts an SRT subtitle file into moviepy text clips and overlays them onto the video.

In [15]:
def srt_to_moviepy_subtitles(srt_file, video_clip):
    subs = pysrt.open(srt_file)
    subtitle_clips = []

    for sub in subs:
        start_time = sub.start.to_time()
        end_time = sub.end.to_time()
        start_seconds = start_time.hour * 3600 + start_time.minute * 60 + start_time.second + start_time.microsecond / 1e6
        end_seconds = end_time.hour * 3600 + end_time.minute * 60 + end_time.second + end_time.microsecond / 1e6
        duration = end_seconds - start_seconds

        # Formatting text to handle newlines properly
        formatted_text = sub.text.replace('\n', ' ')

        # Create a text clip with a black background and white text
        text_clip = (mp.TextClip(formatted_text, fontsize=24, color='yellow', bg_color='black', method='caption', size=(video_clip.w - 20, None))
             .set_start(start_seconds)
             .set_duration(duration)
             .set_position(('center', video_clip.h - 50)))  # Position near the bottom with padding

        # Add the text clip to the list of subtitle clips
        subtitle_clips.append(text_clip)

    return mp.CompositeVideoClip([video_clip] + subtitle_clips)

Explanation:

pysrt.open(srt_file): Opens the subtitle file and parses it.

start_time and end_time: Convert subtitle start and end times to Python datetime objects.

start_seconds and end_seconds: Calculate subtitle start and end times in seconds.

formatted_text: Replace newlines in the subtitle text with spaces.

mp.TextClip: Creates a video clip containing text.

Parameters:
- fontsize: Font size of the text.
- color: Color of the text.
- bg_color: Background color of the text clip.
- method='caption': Text rendering method.
- size: Size of the text clip. Here it is adjusted to fit within the video frame width.
- set_start: Sets the start time of the subtitle clip.
- set_duration: Sets how long the subtitle will be displayed.
- set_position: Sets the position of the text clip on the video frame.

mp.CompositeVideoClip: Combines the original video clip with the subtitle clips.

3.2 Function: burn_subtitles

This function applies the subtitles to a video file and saves the output.

In [16]:
def burn_subtitles(video_file, srt_file, output_file):
    video_clip = mp.VideoFileClip(video_file)
    video_with_subs = srt_to_moviepy_subtitles(srt_file, video_clip)
    video_with_subs.write_videofile(output_file, codec='libx264', audio_codec='aac')

Explanation:

- mp.VideoFileClip(video_file): Loads the video file into a moviepy video clip.
- srt_to_moviepy_subtitles: Calls the previously defined function to add subtitles to the video.
- write_videofile: Writes the final video with subtitles to a file.

Parameters:
 - codec='libx264': Specifies the video codec.
 - audio_codec='aac': Specifies the audio codec.

### Upload input files

In [23]:
from google.colab import files
uploaded_files = files.upload()

# Save uploaded files to disk
video_file_path = None
srt_file_path = None

for file_name in uploaded_files.keys():
    if file_name.endswith('.mp4'):
        video_file_path = file_name
        with open(file_name, 'wb') as f:
            f.write(uploaded_files[file_name])
    elif file_name.endswith('.srt'):
        srt_file_path = file_name
        with open(file_name, 'wb') as f:
            f.write(uploaded_files[file_name])

if video_file_path and srt_file_path:
    print(f"Uploaded video file: {video_file_path}")
    print(f"Uploaded subtitle file: {srt_file_path}")
else:
    raise ValueError("Please upload both a .mp4 video file and a .srt subtitle file.")

Saving input.mp4 to input.mp4
Saving input.srt to input.srt
Uploaded video file: input.mp4
Uploaded subtitle file: input.srt


### Generate and download output

In [24]:
output_file_name = 'output.mp4'
burn_subtitles(video_file_path, srt_file_path, output_file_name)

# Provide a link to download the output file
files.download(output_file_name)

Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output.mp4



t:  99%|█████████▉| 866/873 [00:16<00:00, 70.20it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file input.mp4, 641280 bytes wanted but 0 bytes read,at frame 872/873, at time 34.88/34.90 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready output.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>